In [30]:
from lxml import html
from lxml.cssselect import CSSSelector
import requests
import time
import random
import logging
from listings.scrapers.scraper_util import *
import json

In [3]:
base_url = 'http://www.idealista.it/vendita-case/milano-milano/lista-{}.htm'
max_counter = 628

In [4]:
def list_page_url_generator(base_url, max_counter):
    counter = 1
    while counter <= max_counter:
        yield base_url.format(str(counter))
        counter += 1

In [31]:
def extract_record(elem):
    title_url = [(link.attrib['title'], link.attrib['href'])
                 for link in elem.cssselect('a.item-link')]

    if len(title_url) == 0:
        raise ValueError('empty element')

    titles = [t for t, _ in title_url]
    urls = [u for _, u in title_url]
    ids = [u.split('/')[-2] for u in urls]

    prices = list()
    categories = list()
    phones = [p.text for p in elem.cssselect('span.icon-phone')]

    return {
        'src': 'idealista',
        'id': flatten(ids),
        'title': flatten(titles),
        'url': flatten(urls),
        'price': flatten(prices),
        'category': flatten(categories),
        'datetime': "",
        'location': "",
        'phone': flatten(phones)
    }

In [32]:
def extract_records(url):
    tree = parse(url)
    elements = tree.xpath('//article')
    records = list()
    for elem in elements:
        try:
            records.append(extract_record(elem))
        except ValueError as e:
            logger.info('removed record with content %s', elem.text_content())

    return records

In [33]:
def extract_from_detail_page(record):
    url = record['url']
    tree = parse(url)

    map_details = dict()

    price_down = [e.text_content() for e in tree.cssselect('span.icon-pricedown')]

    map_details['price_down'] = flatten(price_down)

    for kv in tree.xpath('//*[@id="details"]/div'):
        key = flatten([k.text_content() for k in kv.xpath('h2')]).strip()
        value = [v.text_content().strip() for v in kv.xpath('ul/li')]
        map_details[key] = value

    logger.info('extracted details for the url %s', url)

    record['details'] = map_details
    return record

## Example Usage

In [34]:
generator = list_page_url_generator(base_url,max_counter)

In [35]:
url = next(generator)
print(url)

http://www.idealista.it/vendita-case/milano-milano/lista-1.htm


In [37]:
records = extract_records(url)

2016-07-24 11:07:58,929 - scraper - INFO - removed record with content   Pubblicità    


In [40]:
detailed = [extract_from_detail_page(r) for r in records]

2016-07-24 11:09:37,945 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/9074680/
2016-07-24 11:09:43,291 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/9354587/
2016-07-24 11:09:44,668 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/9986437/
2016-07-24 11:09:46,064 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/9390957/
2016-07-24 11:09:50,467 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/9428753/
2016-07-24 11:09:54,917 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/8462870/
2016-07-24 11:09:57,308 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/8815548/
2016-07-24 11:10:02,668 - scraper - INFO - extracted details for the url http://www.idealista.it/immobile/8402932/
2016-07-24 11:10:05,106 - scraper - INFO - extracted details for the url http://

In [38]:
for r in records:
    print(r)
    print()

{'datetime': '', 'price': '', 'title': 'Appartamento in via fetonte, s.n.c, San Siro, Milano', 'location': '', 'id': '9074680', 'url': 'http://www.idealista.it/immobile/9074680/', 'category': '', 'src': 'idealista', 'phone': '0287368104'}

{'datetime': '', 'price': '', 'title': 'Quadrilocale in val cannobina, 6, Baggio, Milano', 'location': '', 'id': '9354587', 'url': 'http://www.idealista.it/immobile/9354587/', 'category': '', 'src': 'idealista', 'phone': '0230464890'}

{'datetime': '', 'price': '', 'title': 'Trilocale in via privata albona, 8, Baggio, Milano', 'location': '', 'id': '9986437', 'url': 'http://www.idealista.it/immobile/9986437/', 'category': '', 'src': 'idealista', 'phone': '0230464890'}

{'datetime': '', 'price': '', 'title': 'Appartamento su due piani in via villapizzone, 26, Varesina-Testori, Milano', 'location': '', 'id': '9390957', 'url': 'http://www.idealista.it/immobile/9390957/', 'category': '', 'src': 'idealista', 'phone': '0230464914'}

{'datetime': '', 'price

## For Testing Rercods

In [39]:
page_tree = parse(url)

In [23]:
articles = page_tree.xpath('//article')

In [22]:
a = articles[0]

In [25]:
highlight_description = a.cssselect('div.item-highlight-phrase')[0].text
print(highlight_description)

San Siro_ Prestigioso appartamento con terrazzi con vista immersa nel verde, per gli amanti dello sport e del relax.


In [26]:
title_url = [(link.attrib['title'], link.attrib['href'])
            for link in a.cssselect('a.item-link')]
print(title_url)

[('Appartamento in via fetonte, s.n.c, San Siro, Milano', 'http://www.idealista.it/immobile/9074680/')]


In [27]:
details = [d.text_content() for d in a.cssselect('span.item-detail')]
print(details)

['5 locali', '270 m2', '3º piano con ascensore']


In [28]:
description = [e.text for e in a.cssselect('p.item-description')]
print(description)

['Grimaldi Agenzia Sempione Brera - Milano: Nell’ambitissimo quartiere San Siro, fronte stadio, zona molto tranquilla e immersa nel verde,...']


In [29]:
phones = [p.text for p in a.cssselect('span.icon-phone')]
print(phones)

['0287368104']


## For Test Detail Record

In [14]:
tree = parse('http://www.idealista.it/immobile/9354587/')

In [15]:
titles = [t.text_content() for t in tree.cssselect('h1.txt-bold')]
print(titles)

['Quadrilocale in vendita in val cannobina, 6, Baggio, Milano']


In [16]:
price_down = [e.text_content() for e in tree.cssselect('span.icon-pricedown')]
print(price_down)

[' È sceso di 15.000 €  (5%) ']


In [17]:
description = [e.text_content() for e in tree.cssselect('div.adCommentsLanguage')]

In [18]:
description

[' "Ad. Via Cabella - In Ottimo e Tranquillo Contesto Medio Signorile in Clinker con Portineria, Campo Da Tennis ed Ampio Verde Condominiale ( La Serenissima ) - Mq. 140 - Nelle Vicinanze di Negozi e Scuole - Ampio Quattro Locali con Cucina Abitabile, Doppi Servizi Finestrati, Due Ripostigli, Balcone, Cantina e Poss. Box Auto - Disponibilità Immediata - € 270.000,00.(Ricerchiamo Immobili Residenziali in vendita in Milano e zone limitrofe anche in nuda proprietà per investimento o in locazione per la nostra clientela). ( Si Effettuano Permute ) - (Cercasi Personale per Ampliamento Organico - Inviare curriculum con foto a royal_casa@virgilio.it )." ']

In [38]:
details = set()
for kv in tree.xpath('//*[@id="details"]/div'):
    key = [k.text_content() for k in kv.xpath('h2')]
    print(key)
    value = [v.text_content() for v in kv.xpath('ul/li')]
    print(value)

['Commento dell’inserzionista']
['  Disponibile in Italiano  "Ad. Via Cabella - In Ottimo e Tranquillo Contesto Medio Signorile in Clinker con Portineria, Campo Da Tennis ed Ampio Verde Condominiale ( La Serenissima ) - Mq. 140 - Nelle Vicinanze di Negozi e Scuole - Ampio Quattro Locali con Cucina Abitabile, Doppi Servizi Finestrati, Due Ripostigli, Balcone, Cantina e Poss. Box Auto - Disponibilità Immediata - € 270.000,00.(Ricerchiamo Immobili Residenziali in vendita in Milano e zone limitrofe anche in nuda proprietà per investimento o in locazione per la nostra clientela). ( Si Effettuano Permute ) - (Cercasi Personale per Ampliamento Organico - Inviare curriculum con foto a royal_casa@virgilio.it )."   ']
['Prezzo']
[' 270.000 euro -  Fai una controproposta  ', ' 1.929 euro/m2 ', ' Calcola mutuo ', ' ', ' 270 euro/mese di spese condominiali ', ' Avvisami se diminuisce il prezzo   Pubblicità    ']
['Vuoi sapere se è economico o caro?']
[' Compra una stima del prezzo ']
['Caratteristi

In [30]:
kv.xpath('h2')

[<Element h2 at 0x10987e7c8>]

In [20]:
details

{'  Disponibile in Italiano  "Ad. Via Cabella - In Ottimo e Tranquillo Contesto Medio Signorile in Clinker con Portineria, Campo Da Tennis ed Ampio Verde Condominiale ( La Serenissima ) - Mq. 140 - Nelle Vicinanze di Negozi e Scuole - Ampio Quattro Locali con Cucina Abitabile, Doppi Servizi Finestrati, Due Ripostigli, Balcone, Cantina e Poss. Box Auto - Disponibilità Immediata - € 270.000,00.(Ricerchiamo Immobili Residenziali in vendita in Milano e zone limitrofe anche in nuda proprietà per investimento o in locazione per la nostra clientela). ( Si Effettuano Permute ) - (Cercasi Personale per Ampliamento Organico - Inviare curriculum con foto a royal_casa@virgilio.it )."   ',
 ' 1.929 euro/m2 ',
 ' 140 m² commerciali ',
 ' 2 bagni ',
 ' 270 euro/mese di spese condominiali ',
 ' 270.000 euro -  Fai una controproposta  ',
 ' 4 locali ',
 ' Avvisami se diminuisce il prezzo   Pubblicità    ',
 ' Buono stato ',
 ' Calcola mutuo ',
 ' Cantina ',
 ' Classe energetica:  (148,84 kWh/m² anno) '

In [43]:
mode = 'force'

In [44]:
mode != 'force'

False